In [ ]:
import tensorflow as tf
import logictensornetworks as ltn
import pandas as pd
import numpy as np
import openpyxl
import commons

In [ ]:
target = pd.read_csv('../Haoming/target_result/GOLD_multiclass.csv', index_col=0)
target.drop(index=225, inplace=True) # no.225 data is lost

train_target = target[target['train'] == 1].drop(['train', 'test'], axis=1)
test_target = target[target['test'] == 1].drop(['train', 'test'], axis=1)

In [ ]:
feature_matrix = pd.read_csv('../Haoming/additional/_feature_matrix_all_sections_.csv', index_col=0)
feature_matrix.fillna(0, inplace=True)

train_feature = feature_matrix.loc[train_target.index]
test_feature = feature_matrix.loc[test_target.index]

In [ ]:
entities = target.columns.values[0:-2]
class_name = []
for i in range(32):
    name  = entities[i]
    class_name.append(name + '_absent')
    class_name.append(name + '_questionable')
    class_name.append(name + '_present')
    if i%2 == 1:
        class_name.append(name + '_unmentioned')

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
batch_size = 64 * strategey.num_replicas_in_sync
ds_train = tf.data.Dataset.from_tensor_slices((train_feature,train_target)).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((test_feature,test_target)).batch(batch_size)
ds_train = strategy.experimental_distribute_dataset(ds_train)
ds_test = strategy.experimental_distribute_dataset(ds_test)

In [ ]:
SIZE=len(feature_matrix.columns)
class MLP(tf.keras.Model):
    def __init__(self, n_classes, hidden_layer_sizes=(SIZE, SIZE, SIZE/2)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
    
    def call(self, inputs):
        x = inputs
        for dense in self.denses:
            x = dense(x)
        return self.dense_class(x)

with strategy.scope():
    logits_model = MLP(len(class_name))
    p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(logits_model,single_label=True))

In [ ]:
classes = {}
for i in range(len(class_name)):
    classes.update({class_name[i]: ltn.Constant(i, trainable=False)})

In [ ]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=6),semantics="exists")

formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

In [ ]:
@tf.function
def axioms(features, labels):
    x = ltn.Variable("x", features)
    x_class = {}
    for i in len(class_name):
        r = i % 7
        a = -99
        if r == 0 or r == 3:
            a = 1
        elif r == 1 or r == 4:
            a = -1
        elif r == 2 or r == 5:
            a = 0
        else:
            a = 3
            
        x_class[class_name[i]] = ltn.Variable(class_name[i], features[labels[entities[r]]==a])
    
    axioms = [
        Forall(x_class[name], p([x_class[name], classes[name]])) for name in class_name
    ]
    for name1 in class_name:
        for name2 in class_name:
            if name1 != name2:
                axioms.append(Forall(x, Not(And(p([x_class[name1], classes[name1]]), p([x_class[name2], classes[name2]])))))
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

In [ ]:
for features, labels in ds_train:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

In [ ]:
with strategy.scope():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    metrics_dict = {
        'train_axiom_sat':tf.keras.metrics.Mean(name='train_axiom_sat'),
        'test_axiom_sat':tf.keras.metrics.Mean(name='test_axiom_sat')
        'train_accuracy': tf.keras.metircs.SparseCategoricalAccuracy(name="train_accuracy"),
        'test_accuracy': tf.keras.metircs.SparseCategoricalAccuracy(name="test_accuracy")
    }

In [ ]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, Training=True)
        loss_value = 1. - sat
    grads = tape.gradient(loss_value, p.trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    sat = axioms(features, labels)
    metrics_dict['train_axiom_sat'](sat)
    predictions = logits_model(features)
    arr = []
    for i in len(class_name):
        r = i % 7
        a = ''
        if r == 0 or r == 3:
            a = '1'
        elif r == 1 or r == 4:
            a = '-1'
        elif r == 2 or r == 5:
            a = '0'
        else:
            a = '3'
        arr.append((class_name==a))
    onehot = tf.stack(arr, aixs=-1)
    metrics_dict['train_accuracy'](1-multilabel_hamming_loss(onehot, predictions, from_logits=True))
    
@tf.function
def test_step(features, labels):
    sat = axioms(features, labels)
    metrics_dict['test_axiom_sat'](sat)
    predictions = logits_model(features)
    arr = []
    for i in len(class_name):
        r = i % 7
        a = ''
        if r == 0 or r == 3:
            a = '1'
        elif r == 1 or r == 4:
            a = '-1'
        elif r == 2 or r == 5:
            a = '0'
        else:
            a = '3'
        arr.append((class_name==a))
    onehot = tf.stack(arr, aixs=-1)
    metrics_dict['test_accuracy'](1-multilabel_hamming_loss(onehot, predictions, from_logits=True))
    

In [ ]:
@tf.function
def distributed_train_step(features, labels):
    strategy.run(train_step, args=(features, labels,))

@tf.function
def distributed_test_step(features, labels):
    strategy.run(test_step, args=(features, labels,))

In [ ]:
EPOCHS = 200

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    distributed_train_step,
    distributed_test_step,
    csv_path='',
    track_metrics=20
)